# Import Statements

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Read Files

In [2]:
base_path = 'Dataset/ml-100k/'

In [3]:
cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
train_1 = pd.read_csv(base_path+'u5.base', sep='\t', names=cols, encoding='latin-1')
test_1 = pd.read_csv(base_path+'u5.test', sep='\t', names=cols, encoding='latin-1')

# Train Matrix

In [4]:
train_matrix_1 = np.zeros((943,1682))

In [5]:
for i in range(80000):
    train_matrix_1[train_1.iloc[i,0]-1][train_1.iloc[i,1]-1] = train_1.iloc[i,2]

# Test Matrix

In [6]:
test_matrix_1 = np.zeros((943,1682))

In [7]:
for i in range(20000):
    test_matrix_1[test_1.iloc[i,0]-1][test_1.iloc[i,1]-1] = test_1.iloc[i,2]

In [8]:
train_matrix_1.shape

(943, 1682)

# Mean Rating by User

In [9]:
mean_rating_1 = np.average(train_matrix_1, axis=1, weights=(train_matrix_1 > 0))

In [10]:
mean_rating_1.shape

(943,)

# Mean Rating Of Item

In [11]:
mean_rating_item_1 = np.zeros((1682,))

In [12]:
for i in range(1682):
    summ = 0
    counter = 0
    for j in range(943):
        if train_matrix_1[j][i]>0:
            summ += train_matrix_1[j][i]
            counter += 1
    if counter>0:
        mean_rating_item_1[i] = summ/counter

# Variance Of Item

In [13]:
variance_rating_item_1 = np.zeros((1682,))

In [14]:
for i in range(1682):
    if mean_rating_item_1[i]==0:
        continue
    summ = 0
    counter = 0
    for j in range(943):
        if train_matrix_1[j][i]>0:
            summ += (train_matrix_1[j][i]-mean_rating_item_1[i])*(train_matrix_1[j][i]-mean_rating_item_1[i])
            counter += 1
    if counter-1>0:
        variance_rating_item_1[i] = summ/(counter-1)

# Number of co-rated items matrix

In [15]:
co_rated_matrix_1 = np.zeros((943,943))

In [16]:
for i in tqdm(range(943)):
    for j in range(i+1,943):
        for k in range(1682):
            if(train_matrix_1[i][k]>0 and train_matrix_1[j][k]>0):
                co_rated_matrix_1[i][j]+=1
                co_rated_matrix_1[j][i]+=1

100%|████████████████████████████████████████████████████████████████████████████████| 943/943 [07:26<00:00,  2.11it/s]


# User Cosine similarity Matrix

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
cosine_similarity_1 = cosine_similarity(train_matrix_1)

# Item Cosine similarity Matrix

In [19]:
cosine_similarity_item = cosine_similarity(train_matrix_1.T)

# User Based Prediction

## Fold 1 k = 10, 20, 30, 40, 50

In [20]:
for k in [-11,-21,-31,-41,-51]:
    total_predictions_1 = 0
    total_absolute_error_1 = 0
    for a in tqdm(range(943)):
        # calculating top k neighbours of active user 'a'
        # k = actual k + 1 since argpartition will return 'a' itself as well as cosine-sim(a,a) is 1
        neighbours = np.argpartition(cosine_similarity_1[a], k)[k:]
        for j in range(1682):
            if test_matrix_1[a][j]==0:
                continue
            summation=0
            normalizing_factor=0
            for i in neighbours:
                if i==a:
                    continue
                if train_matrix_1[i][j]==0:
                    continue
                summation += cosine_similarity_1[a][i]*(train_matrix_1[i][j]-mean_rating_1[i])
                normalizing_factor += abs(cosine_similarity_1[a][i])
            if normalizing_factor==0:
                continue
            predicted_rating_a_j_1 = round(mean_rating_1[a] + (summation/normalizing_factor))
            total_absolute_error_1 += abs(predicted_rating_a_j_1 - test_matrix_1[a][j])
            total_predictions_1 += 1
    coverage_1 = (total_predictions_1/20000)*100
    mean_absolute_error_1 = total_absolute_error_1/total_predictions_1
    print('Coverage: {} {}'.format(coverage_1, '%'))
    print('Mean Absolute Error Fold 1 k={}: {}'.format(-1*k-1,mean_absolute_error_1))

100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:01<00:00, 579.94it/s]


Coverage: 90.31 %
Mean Absolute Error Fold 1 k=10: 0.7679658952496955


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 453.35it/s]


Coverage: 95.42500000000001 %
Mean Absolute Error Fold 1 k=20: 0.7340319622740372


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:01<00:00, 493.07it/s]


Coverage: 97.195 %
Mean Absolute Error Fold 1 k=30: 0.7256031688872885


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 386.77it/s]


Coverage: 97.935 %
Mean Absolute Error Fold 1 k=40: 0.7176188288150304


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 365.95it/s]

Coverage: 98.33 %
Mean Absolute Error Fold 1 k=50: 0.7123461812264823


## Fold 1 k = 10, 20, 30, 40, 50 Significance Weighting

In [21]:
for k in [-11,-21,-31,-41,-51]:
    total_predictions_1_sw = 0
    total_absolute_error_1_sw = 0
    for a in tqdm(range(943)):
        # calculating top k neighbours of active user 'a'
        # k = actual k + 1 since argpartition will return 'a' itself as well as cosine-sim(a,a) is 1
        threshold = -1*k-1
        neighbours = np.argpartition(cosine_similarity_1[a], k)[k:]
        for j in range(1682):
            if test_matrix_1[a][j]==0:
                continue
            summation=0
            normalizing_factor=0
            for i in neighbours:
                if i==a:
                    continue
                if train_matrix_1[i][j]==0:
                    continue
                significance_weight = 0
                if co_rated_matrix_1[a][i]<threshold:
                    significance_weight = co_rated_matrix_1[a][i]/threshold
                else:
                    significance_weight = 1
                summation += cosine_similarity_1[a][i]*significance_weight*(train_matrix_1[i][j]-mean_rating_1[i])
                normalizing_factor += abs(cosine_similarity_1[a][i]*significance_weight)
            if normalizing_factor==0:
                continue
            predicted_rating_a_j_1_sw = round(mean_rating_1[a] + (summation/normalizing_factor))
            total_absolute_error_1_sw += abs(predicted_rating_a_j_1_sw - test_matrix_1[a][j])
            total_predictions_1_sw += 1
    coverage_1_sw = (total_predictions_1_sw/20000)*100
    mean_absolute_error_1_sw = total_absolute_error_1_sw/total_predictions_1_sw
    print('Coverage_sw: {} {}'.format(coverage_1_sw, '%'))
    print('Mean Absolute Error Fold 1 k=10 sw: {}'.format(mean_absolute_error_1_sw))

100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:01<00:00, 692.24it/s]


Coverage_sw: 90.31 %
Mean Absolute Error Fold 1 k=10 sw: 0.7674122467057911


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 459.28it/s]


Coverage_sw: 95.42500000000001 %
Mean Absolute Error Fold 1 k=10 sw: 0.7335603877390621


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 406.58it/s]


Coverage_sw: 97.195 %
Mean Absolute Error Fold 1 k=10 sw: 0.7260661556664437


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:02<00:00, 327.27it/s]


Coverage_sw: 97.935 %
Mean Absolute Error Fold 1 k=10 sw: 0.7176698830857201


100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:03<00:00, 277.16it/s]

Coverage_sw: 98.33 %
Mean Absolute Error Fold 1 k=10 sw: 0.7138208074849995


# Item Based Prediction

## Fold 1 k = 10, 20, 30, 40, 50

In [22]:
for k in [-11,-21,-31,-41,-51]:
    total_predictions_1 = 0
    total_absolute_error_1 = 0
    for j in tqdm(range(1682)):
        # calculating top k neighbours of item 'j'
        # active user 'a'
        # k = actual k + 1 since argpartition will return 'a' itself as well as cosine-sim(j,j) is 1
        neighbours = np.argpartition(cosine_similarity_item[j], k)[k:]
        for a in range(943):
            if test_matrix_1[a][j]==0:
                continue
            summation=0
            normalizing_factor=0
            for i in neighbours:
                if i==j:
                    continue
                if train_matrix_1[a][i]==0:
                    continue
                summation += cosine_similarity_item[j][i]*(train_matrix_1[a][i]-mean_rating_item_1[i])
                normalizing_factor += abs(cosine_similarity_item[j][i])
            if normalizing_factor==0:
                continue
            predicted_rating_a_j_1 = round(mean_rating_item_1[j] + (summation/normalizing_factor))
            total_absolute_error_1 += abs(predicted_rating_a_j_1 - test_matrix_1[a][j])
            total_predictions_1 += 1
    coverage_1 = (total_predictions_1/20000)*100
    mean_absolute_error_1 = total_absolute_error_1/total_predictions_1
    print('Coverage: {} {}'.format(coverage_1, '%'))
    print('Mean Absolute Error Fold 1 k={}: {}'.format(-1*k-1,mean_absolute_error_1))

100%|████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:01<00:00, 1165.32it/s]


Coverage: 88.395 %
Mean Absolute Error Fold 1 k=10: 0.7325640590531138


100%|████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:01<00:00, 1047.43it/s]


Coverage: 94.66499999999999 %
Mean Absolute Error Fold 1 k=20: 0.7114561876089368


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:01<00:00, 898.05it/s]


Coverage: 96.705 %
Mean Absolute Error Fold 1 k=30: 0.7016700274029264


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:02<00:00, 803.15it/s]


Coverage: 97.765 %
Mean Absolute Error Fold 1 k=40: 0.6948294379379123


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:02<00:00, 685.05it/s]

Coverage: 98.385 %
Mean Absolute Error Fold 1 k=50: 0.6910606291609493


## Fold 1 k = 10, 20, 30, 40, 50 Variance Weighting

In [25]:
for k in [-11,-21,-31,-41,-51]:
    total_predictions_1 = 0
    total_absolute_error_1 = 0
    for j in tqdm(range(1682)):
        # calculating top k neighbours of item 'j'
        # active user 'a'
        # k = actual k + 1 since argpartition will return 'a' itself as well as cosine-sim(j,j) is 1
        neighbours = np.argpartition(cosine_similarity_item[j], k)[k:]
        for a in range(943):
            if test_matrix_1[a][j]==0:
                continue
            summation=0
            normalizing_factor=0
            for i in neighbours:
                if i==j:
                    continue
                if train_matrix_1[a][i]==0:
                    continue
                if(max(variance_rating_item_1[neighbours])==min(variance_rating_item_1[neighbours])):
                    variance_weight = variance_rating_item_1[j]-min(variance_rating_item_1[neighbours])
                else:
                    variance_weight = (variance_rating_item_1[j]-min(variance_rating_item_1[neighbours]))/(max(variance_rating_item_1[neighbours])-min(variance_rating_item_1[neighbours]))
                summation += cosine_similarity_item[j][i]*variance_weight*(train_matrix_1[a][i]-mean_rating_item_1[i])
                normalizing_factor += abs(cosine_similarity_item[j][i]*variance_weight)
            if normalizing_factor==0:
                continue
            predicted_rating_a_j_1 = round(mean_rating_item_1[j] + (summation/normalizing_factor))
            total_absolute_error_1 += abs(predicted_rating_a_j_1 - test_matrix_1[a][j])
            total_predictions_1 += 1
    coverage_1 = (total_predictions_1/20000)*100
    mean_absolute_error_1 = total_absolute_error_1/total_predictions_1
    print('Coverage: {} {}'.format(coverage_1, '%'))
    print('Mean Absolute Error Fold 1 k={}: {}'.format(-1*k-1,mean_absolute_error_1))

100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:02<00:00, 638.18it/s]


Coverage: 80.985 %
Mean Absolute Error Fold 1 k=10: 0.7365561523738964


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:04<00:00, 367.63it/s]


Coverage: 91.58500000000001 %
Mean Absolute Error Fold 1 k=20: 0.7123983185019381


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:06<00:00, 243.86it/s]


Coverage: 95.295 %
Mean Absolute Error Fold 1 k=30: 0.7020305367542893


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:09<00:00, 177.14it/s]


Coverage: 96.67 %
Mean Absolute Error Fold 1 k=40: 0.6949415537395263


100%|█████████████████████████████████████████████████████████████████████████████| 1682/1682 [00:12<00:00, 130.99it/s]

Coverage: 97.555 %
Mean Absolute Error Fold 1 k=50: 0.6905847983188971
